In [1]:
from datetime import datetime as dt
import pandas as pd
import re

In [ ]:
rawdata = pd.read_excel('Example.xlsx',header=None)
rawdata

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Sales Period: Mar-01-2024 - Mar-17-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN,03-Mar - 2024,NaN
2,NaN,NaN,Branch_Code:1001,NaN,Branch_Code:1002,NaN,Branch_Code:1004,NaN,Branch_Code:1015,NaN,Branch_Code:1024,NaN,Total,NaN
3,Article,Description,Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB),Sales (Qty.),Sales (THB)
4,154312001,Sock,5,300,4,240,1,60,NaN,NaN,NaN,NaN,10,600
5,154312002,T-shirt,1,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,40
6,NaN,NaN,6,340,4,240,1,60,NaN,NaN,NaN,NaN,11,640
7,154312051,shoe,1,100,NaN,NaN,NaN,NaN,4,400,NaN,NaN,5,500
8,154312052,denture,9,1800,3,600,1,200,1,200,7,1400,21,4200
9,NaN,NaN,10,1900,3,600,5,200,5,600,7,1400,26,4700


In [5]:
#Find Date Report
date_index = rawdata.index[rawdata.iloc[:,0].astype(str).apply(lambda raw : bool(re.findall('Period',raw)))]
rawdate_str = rawdata.iloc[:,0].iloc[date_index].pop(0)
date_str = '-'.join(rawdate_str.split('-')[-3:]).replace(' ','')
date_report = dt.strptime(date_str,'%b-%d-%Y')
date_report

datetime.datetime(2024, 3, 17, 0, 0)

In [6]:
#Rename Columns and Select values
columns_index = rawdata.index[rawdata.iloc[:,0].astype(str).apply(lambda raw: bool(re.findall('^Article',raw)))].to_list().pop()
rawdata.iloc[columns_index-1] = rawdata.iloc[columns_index-1].ffill().astype(str)

columns_sale = [i for i,col in enumerate (rawdata.iloc[columns_index].apply(lambda raw: bool(re.findall('^Sales',str(raw)))).to_list()) if col].pop(0)
rawdata.iloc[columns_index][columns_sale:] = rawdata.iloc[columns_index-1][columns_sale:].apply(lambda raw: raw.replace('Branch_Code:','').replace(' ',''))+'/'+rawdata.iloc[columns_index][columns_sale:].apply(lambda raw:  re.findall(r'\((.*?)\)',raw.replace('.',''))[0])

rawdata.columns = rawdata.iloc[columns_index].to_list()
rawdata.drop(columns=['Description'],axis=1,inplace=True)
rawdata.drop(index=range(columns_index+1),axis=1,inplace=True)
rawdata = rawdata[rawdata['Article'].apply(lambda d: isinstance(d,int))]
rawdata

,Article,1001/Qty,1001/THB,1002/Qty,1002/THB,1004/Qty,1004/THB,1015/Qty,1015/THB,1024/Qty,1024/THB,Total/Qty,Total/THB
4,154312001,5,300,4,240,1,60,NaN,NaN,NaN,NaN,10,600
5,154312002,1,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,40
7,154312051,1,100,NaN,NaN,NaN,NaN,4,400,NaN,NaN,5,500
8,154312052,9,1800,3,600,1,200,1,200,7,1400,21,4200


In [10]:
#Unpivot
data_melt = pd.melt(rawdata, id_vars=['Article'],var_name='type',value_name='value')
data_melt = data_melt[(data_melt['value'].notna()) & (~ data_melt['type'].str.contains('Total', na=False))]
data_melt[['branch_id', 'unit']] = data_melt['type'].apply(lambda x: pd.Series(x.split('/')))
data_melt

,Article,type,value,branch_id,unit
0,154312001,1001/Qty,5,1001,Qty
1,154312002,1001/Qty,1,1001,Qty
2,154312051,1001/Qty,1,1001,Qty
3,154312052,1001/Qty,9,1001,Qty
4,154312001,1001/THB,300,1001,THB
5,154312002,1001/THB,40,1001,THB
6,154312051,1001/THB,100,1001,THB
7,154312052,1001/THB,1800,1001,THB
8,154312001,1002/Qty,4,1002,Qty
11,154312052,1002/Qty,3,1002,Qty


In [12]:
#Pivot
data = data_melt.pivot(index=['Article', 'branch_id'], columns='unit', values='value').reset_index()
data.columns.name = None
data['sale_date'] = date_report
new_columns = {
    "Article":"shop_barcode",
    'Qty':'qty',
    'THB':'amount'
}
data.rename(columns=new_columns,inplace=True)
data

C:\Users\intrust\AppData\Local\Temp\ipykernel_30764\825639394.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  data = data_melt.pivot(index=['Article', 'branch_id'], columns='unit', values='value').reset_index()


,shop_barcode,branch_id,qty,amount,sale_date
0,154312001,1001,5,300,2024-03-17
1,154312001,1002,4,240,2024-03-17
2,154312001,1004,1,60,2024-03-17
3,154312002,1001,1,40,2024-03-17
4,154312051,1001,1,100,2024-03-17
5,154312051,1015,4,400,2024-03-17
6,154312052,1001,9,1800,2024-03-17
7,154312052,1002,3,600,2024-03-17
8,154312052,1004,1,200,2024-03-17
9,154312052,1015,1,200,2024-03-17
